### POS tagging with maximum entropy models (10 pts)

In this task you will build a maximum entropy model for part-of-speech tagging. As the name suggests, our problem is all about converting a sequence of words into a sequence of part-of-speech tags. 
<img src=https://i.stack.imgur.com/6pdIT.png width=320>


__Your man goal:__ implement the model from [the article you're given](W96-0213.pdf).

Unlike previous tasks, this one gives you greater degree of freedom and less automated tests. We provide you with programming interface but nothing more.

__A piece of advice:__ there's a lot of objects happening here. If you don't understand why some object is needed, find `def train` function and see how everything is linked together.


### Part I: reading input data

In [ ]:
import collections

In [ ]:
# Data types:
# Word: str
# Sentence: list of str
TaggedWord = collections.namedtuple('TaggedWord', ['text', 'tag'])
# TaggedSentence: list of TaggedWord
# Tags: list of TaggedWord
# TagLattice: list of Tags

def read_tagged_sentences(path):
    """
    Read tagged sentences from CoNLL-U file and return array of TaggedSentence (array of lists of TaggedWord).
    """
    tagged_sentences = []
    
    with open(path, encoding='utf-8') as reader:
        tagged_sentence = []
        for line in reader.readlines():
            if line.startswith('#'):
                continue
            elif line.startswith('\n'):
                tagged_sentences.append(tagged_sentence)
                tagged_sentence = []
            else:
                line = line.split('\t')
                tagged_sentence.append(TaggedWord(text=line[1], tag=line[3]))
    
    return tagged_sentences

def write_tagged_sentence(tagged_sentence, f):
    """
    Write tagged sentence in CoNLL-U format to file-like object f.
    """
    words, tags = zip(*tagged_sentence)
    
    f.write(' '.join(words))
    f.write(' '.join(tags))
    

def read_tags(path):
    """
    Read a list of possible tags from file and return the list.
    """
    with open(path) as f:
        tags = f.readlines()
        
    return list(map(str.strip, tags))

### Part II: evaluation

We want you to estimate tagging quality by a simple accuracy: a fraction of tag predictions that turned out to be correct - averaged over the entire training corpora.

In [ ]:
# Data types:
TaggingQuality = collections.namedtuple('TaggingQuality', ['acc'])

def tagging_quality(ref, out):
    """
    Compute tagging quality and reutrn TaggingQuality object.
    """
    pront(ref[0], out[0])
    nwords = 0
    ncorrect = 0
    import itertools
    for ref_sentence, out_sentence in itertools.zip_longest(ref, out):
        for ref_word, out_word in itertools.zip_longest(ref_sentence, out_sentence):
            nwords += 1
            ncorrect += ref_word.tag == out_word.tag
    return TaggingQuality(ncorrect / nwords)

### Part III: Value and Update

In order to implement two interlinked data structures: 
* __Value__ - a class that holds POS tagger's parameters. Basically an array of numbers
* __Update__ - a class that stores updates for Value

In [ ]:
import numpy as np

class Value:
    def __init__(self, n):
        """
        Dense object that holds parameters.
        :param n: array length
        """
        self.params = np.random.normal(0, 0.01, n)

    def dot(self, update):
        return np.dot(self.params[update.positions], update.values)

    def assign(self, other):
        """
        self = other
        other is Value.
        """
        self.params = other.params

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff is float.
        """
        self.params *= coef

    def assign_madd(self, x, coeff):
        """
        self = self + x * coeff
        x can be either Value or Update.
        coeff is float.
        """
        if isinstance(x, Value):
            self.params += x.params * coeff
        elif isinstance(x, Update):
            self.params[x.positions.astype(int)] += x.values * coeff
        else:
            raise NotImplementedError
            
    def __len__(self):
        return self.params.shape[0]


class Update:
    """
    Sparse object that holds an update of parameters.
    """

    def __init__(self, positions=[], values=[]):
        """
        positions: array of int
        values: array of float
        """
        self.positions = np.array(positions, dtype=np.int64)
        self.values = np.array(values)

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff: float
        """
        self.values *= coeff

    def assign_madd(self, update, coeff):
        """
        self = self + update * coeff
        coeff: float
        """
        self.positions = np.concatenate((self.positions, update.positions))
        self.values = np.concatenate((self.values, update.values * coeff))

### Part IV: Maximum Entropy POS Tagger
_step 1 - draw an oval; step 2 - draw the rest of the owl (c)_

In this secion you will implement a simple linear model to predict POS tags.
Make sure you [read the article](W96-0213.pdf) before you proceed.

In [ ]:
# Data Types:
Features = Update
Hypo = collections.namedtuple('Hypo', ['prev', 'pos', 'tagged_word', 'score'])
# prev: previous Hypo
# pos: position of word (0-based)
# tagged_word: tagging of source_sentence[pos]
# score: sum of scores over edges

TaggerParams = collections.namedtuple('FeatureParams', [
    'src_window',
    'dst_order',
    'max_suffix',
    'beam_size',
    'nparams'
    ])

def h(x):
    """
    Compute CityHash of any object.
    Can be used to construct features.
    """
    import mmh3
    return mmh3.hash64(repr(x))[0]

In [ ]:
class LinearModel:
    """
    A thing that computes score and gradient for given features.
    """

    def __init__(self, n):
        self._params = Value(n)

    def params(self):
        return self._params

    def score(self, features):
        """
        features: Update
        """
        return self._params.dot(features)

    def gradient(self, features, score):
        return features

In [ ]:
import re

class FeatureComputer:
    def __init__(self, tagger_params, source_sentence):
        self.tagger_params = tagger_params
        self.source_sentence = source_sentence

    def compute_features(self, hypo):
        """
        Compute features for a given Hypo and return Update.
        """
        word = hypo.tagged_word.text.lower()
        features = [h((hypo.tagged_word.tag, word))]
        
        for length in range(1, self.tagger_params.max_suffix + 1):
            if length <= len(word):
                features.extend([
                    h((hypo.tagged_word.tag, 'suffix_{}_{}'.format(length, word[-length:]))),
                    h((hypo.tagged_word.tag, 'prefix_{}_{}'.format(length, word[:length])))
                ])
                features.extend([
                    h((hypo.tagged_word.tag, 'suffix_{}_{}'.format(length, ""))),
                    h((hypo.tagged_word.tag, 'prefix_{}_{}'.format(length, "")))
                ])
                
            features.extend([
                h((hypo.tagged_word.tag, 'is_capital_{}'.format(
                    hypo.tagged_word.text == hypo.tagged_word.text.capitalize()))),
                h((hypo.tagged_word.tag, 'is_upper_{}'.format(hypo.tagged_word.text.isupper()))),
                h((hypo.tagged_word.tag, 'is_number_{}'.format(None != re.search('\d', word)))),
                h((hypo.tagged_word.tag, 'has_hyphen_{}'.format(None != re.search('\-', word)))),
            ])
            
            new_hypo = hypo
            prev_tags = []
            for tag_ind in range(self.tagger_params.dst_order):
                new_hypo = new_hypo.prev if new_hypo else None
    
                if new_hypo is None:
                    prev_tags.append(None)
                else:
                    prev_tags.append(new_hypo.tagged_word.tag)
                
                features.append(h((hypo.tagged_word.tag, 'prev_{}_tags_{}'.format(tag_ind+1, '_'.join(map(str, prev_tags))))))
                
            delta = list(range(hypo.pos-self.tagger_params.src_window, hypo.pos+self.tagger_params.src_window+1))
            delta = list(filter(lambda x: (x != hypo.pos), delta))
    
            # delta_filtered = list(filter(lambda x: (x>=0 and x<len(self.source_sentence)
            #                                and x != hypo.pos), delta))
            
            for ind in delta:
                if ind >= 0:
                    try:
                        word_orig = self.source_sentence[ind]
                        word = self.source_sentence[ind].lower()
                        if (ind == 0):
                            if word_orig.isupper():
                                word = word_orig
                        
                    except IndexError:
                        word = 'OOS'    
                else:
                    word = 'OOS'
                    # out of sentence
                features.append(h((hypo.tagged_word.tag, 'ind_{}_{}'.format(ind, word))))
                
            features = [h(elem) % self.tagger_params.nparams for elem in features]

            # t_{i-1} - предыдущий тег
            # t_{i-2}t_{i-1} - два предыдущий тега
            # слова в окне - w_{i-1} - два вперёд и два назадъ
            return Update(positions=np.array(features), values=np.ones(len(features)))

### Part V: Beam search

We can find the most likely tagging approximately using Beam Search. As everything else, it comes with a separate interface.

In [ ]:
class BeamSearchTask:
    """
    An abstract beam search task. Can be used with beam_search() generic
    function.
    """

    def __init__(self, tagger_params, source_sentence, model, tags):
        self._tagger_params = tagger_params
        self._source_sentence = source_sentence
        self._model = model
        self._tags = tags
        self._feature_computer = FeatureComputer(tagger_params, source_sentence)

    def total_num_steps(self):
        """
        Number of hypotheses between beginning and end (number of words in
        the sentence).
        """
        return len(self._source_sentence)

    def beam_size(self):
        return self._tagger_params.beam_size

    def _get_next_hypo(self, hypo, tag):
        pos = 0 if not hypo else hypo.pos + 1
        tagged_word = TaggedWord(self._source_sentence[pos], tag)
        new_hypo = Hypo(hypo, pos, tagged_word, 0)
        features = self._feature_computer.compute_features(new_hypo)
        score = self._model.score(features)
        
        if hypo:
            score += hypo.score
            
        return new_hypo._replace(score=score)

    def expand(self, hypo):
        """
        Given Hypo, return a list of its possible expansions.
        'hypo' might be None -- return a list of initial hypos then.
        
        Compute hypotheses' scores inside this function!
        """
        return [self._get_next_hypo(hypo, tag) for tag in self._tags]

    def recombo_hash(self, hypo):
        """
        If two hypos have the same recombination hashes, they can be collapsed
        together, leaving only the hypothesis with a better score.
        """
        result = []
        
        for i in range(self._tagger_params.dst_order):
            result.append(hypo.tagged_word.tag)
            hypo = hypo.prev
            
        return result


def beam_search(beam_search_task):
    """
    Return list of stacks.
    Each stack contains several hypos, sorted by score in descending 
    order (i.e. better hypos first).
    """
    if not beam_search_task.total_num_steps():
        return [[]]

    beam_size = beam_search_task.beam_size()
    first_stack = beam_search_task.expand(None)
    stacks = [sorted(first_stack, key=lambda x: -x.score)[:beam_size]]

    for _ in range(1, beam_search_task.total_num_steps()):
        stack = []
        for hypo in stacks[-1]:
            stack += beam_search_task.expand(hypo)

        stacks.append(sorted(stack, key=lambda x: -x.score)[:beam_size])

    return stacks

In [ ]:
def tag_sentence(sentence, tagger_params, model, tags):
        if not sentence:
            return []
        sentence = [word.text for word in sentence]
        bst = BeamSearchTask(tagger_params, sentence, model, tags)
        stacks = beam_search(bst)
        hypo = stacks[-1][0]
        result = []
        for word in reversed(sentence):
            result.append(TaggedWord(word, hypo.tagged_word.tag))
            hypo = hypo.prev
        return list(reversed(result))

def tag_sentences(dataset, tagger_params, model, tags):
    """
    Main predict function.
    Tags all sentences in dataset. Dataset is a list of TaggedSentence; while 
    tagging, ignore existing tags.
    """
    
    return [tag_sentence(sentence, tagger_params, model, tags) for sentence in dataset]

### Part VI: Optimization objective and algorithm

Once we defined our model and inference algorithm, we can define an optimization task: an object that computes loss function and its gradients w.r.t. model parameters.

In [ ]:
class OptimizationTask:
    """
    Optimization task that can be used with sgd().
    """

    def params(self):
        """
        Parameters which are optimized in this optimization task.
        Return Value.
        """
        raise NotImplementedError()

    def loss_and_gradient(self, golden_sentence):
        """
        Return (loss, gradient) on a specific example.

        loss: float
        gradient: Update
        """
        raise NotImplementedError()


# class UnstructuredPerceptronOptimizationTask(OptimizationTask):
#     def __init__(self, ...):
#         <YOUR CODE>

#     def params(self):
#         <YOUR CODE>

#     def loss_and_gradient(self, golden_sentence):
#         <YOUR CODE>


class StructuredPerceptronOptimizationTask(OptimizationTask):
    def __init__(self, tagger_params, tags):
        self.tagger_params = tagger_params
        self.model = LinearModel(tagger_params.nparams)
        self.tags = tags
        print(tags)

    def params(self):
        return self.model.params()

    def loss_and_gradient(self, golden_sentence):
        # Do beam search.
        beam_search_task = BeamSearchTask(
            self.tagger_params,
            [golden_tagged_word.text for golden_tagged_word in golden_sentence],
            self.model,
            self.tags
            )
        stacks = beam_search(beam_search_task)

        # Compute chain of golden hypos (and their scores!).
        golden_hypo = None
        golden_source_sentence = [word.text for word in golden_sentence]
        feature_computer = FeatureComputer(self.tagger_params, golden_source_sentence)
        
        max_violation = 0
        for i, word in enumerate(golden_sentence):
            new_golden_hypo = Hypo(golden_hypo, i, word, 0)
            score = self.model.score(feature_computer.compute_features(new_golden_hypo))
            if golden_hypo is not None:
                score += golden_hypo.score

            golden_hypo = new_golden_hypo._replace(score=score)

            violation = stacks[i][-1].score - score
            if violation > max_violation:
                max_violation = violation
                rival_head = stacks[i][0]
                golden_head = golden_hypo

        # Find where to update.
        if max_violation == 0:
            rival_head = stacks[-1][0]
            golden_head = golden_hypo

        # Compute gradient.
        grad = Update()
        while golden_head and rival_head:
            for head, multiplyer in zip([rival_head, golden_head], [1, -1]):
                features = feature_computer.compute_features(head)
                add_grad = self.model.gradient(features, score=None)
                grad.assign_madd(add_grad, multiplyer)

            golden_head = golden_head.prev
            rival_head = rival_head.prev

        return grad


### Part VII: optimizer

By this point we can define a model with parameters $\theta$ and a problem that computes gradients $ \partial L \over \partial \theta $ w.r.t. model parameters.

Optimization is performed by gradient descent: $ \theta := \theta - \alpha {\partial L \over \partial \theta} $

In order to speed up training, we use stochastic gradient descent that operates on minibatches of data.

In [ ]:
SGDParams = collections.namedtuple('SGDParams', [
    'epochs',
    'learning_rate',
    'minibatch_size',
    'average' # bool or int
    ])


def make_batches(dataset, minibatch_size):
    """
    Make list of batches from a list of examples.
    """
    batches = []
    for begin_ind in range(0, len(dataset), minibatch_size):
        batches.append(dataset[begin_ind:begin_ind + minibatch_size])

    return batches


def sgd(sgd_params, optimization_task, dataset, after_each_epoch_fn):
    """
    Run (averaged) SGD on a generic optimization task. Modify optimization
    task's parameters.

    After each epoch (and also before and after the whole training),
    run after_each_epoch_fn().
    """
    after_each_epoch_fn()

    if sgd_params.average:
        params_sum = Value(len(optimization_task.params()))
        added_cnt = 0

    for _ in tqdm_notebook(range(sgd_params.epochs)):
        batches = make_batches(dataset, sgd_params.minibatch_size)
        for ind, batch in enumerate(batches):
            grad = Update()
            for sent in batch:
                sent_grad = optimization_task.loss_and_gradient(sent)
                grad.assign_madd(sent_grad, 1)
            optimization_task.params().assign_madd(grad, -sgd_params.learning_rate)
            if sgd_params.average and ind % sgd_params.average == sgd_params.average - 1:
                params_sum.assign_madd(optimization_task.params(), 1)
                added_cnt += 1
        after_each_epoch_fn()

    if sgd_params.average:
        params_sum.assign_mul(1 / added_cnt)
        optimization_task.params().assign(params_sum)
        after_each_epoch_fn()


### Part VIII: Training loop

The train function combines everthing you used below to get new 

In [ ]:
import os
import pprint
import pickle

def train(
    tags='./data/tags',
    train_dataset='./data/en-ud-train.conllu',
    dev_dataset='./data/en-ud-dev.conllu',
    model='./model.npz',
    
    sgd_epochs=15,
    sgd_learning_rate=0.05,
    sgd_minibatch_size=32,
    sgd_average=True,
    
    # Number of context tags in output tagging to use for features
    tagger_src_window=2,
    
    # Number of context tags in output tagging to use for features
    tagger_dst_order=3,
    
    # Maximal number of prefix/suffix letters to use for features
    tagger_max_suffix=4,
    
    # Width for beam search (0 means unstructured)
    beam_size=4,
    
    # Parameter vector size (for hashing)
    nparams= 2 * 22,
):
    """ Train a pos-tagger model and save it's parameters to :model: """

    with open(tags) as f:
        tags = list(map(str.strip, f.readlines()))
    # Beam size.
    optimization_task_cls = StructuredPerceptronOptimizationTask
    if beam_size == 0:
        beam_size = 1
        optimization_task_cls = UnstructuredPerceptronOptimizationTask

    train_dataset = read_tagged_sentences(train_dataset)
    dev_dataset = read_tagged_sentences(dev_dataset)
    params = None
    if os.path.exists(model):
        params = pickle.load(open(model, 'rb'))
    sgd_params = SGDParams(
        epochs=sgd_epochs,
        learning_rate=sgd_learning_rate,
        minibatch_size=sgd_minibatch_size,
        average=sgd_average
        )
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=nparams
        )

    # Load optimization task
    optimization_task = StructuredPerceptronOptimizationTask(tagger_params, tags)
    if params is not None:
        print('\n\nLoading parameters from %s\n\n' % model)
        optimization_task.params().assign(params)

    # Validation.
    def after_each_epoch_fn():
        lm = LinearModel(tagger_params.nparams)
        lm.params().assign(optimization_task.params())
        tagged_sentences = tag_sentences(dev_dataset, tagger_params, lm, tags)
        q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dev_dataset))
        print()
        print(q)
        print()

        # Save parameters.
        print('\n\nSaving parameters to %s\n\n' % model)
        pickle.dump(optimization_task.params(), open(model, 'wb'))

    # Run SGD.
    sgd(sgd_params, optimization_task, train_dataset, after_each_epoch_fn)


In [ ]:
# train a model with default params
train(model='./model_beam_4.npz')

### Part IX: Evaluate the trained model

In [ ]:
def test(
    tags='./data/tags',
    dataset='./data/en-ud-dev.conllu',
    model='./model.npz',
    
    # model and inference params; see train for their description
    tagger_src_window=2,
    tagger_dst_order=3,
    tagger_max_suffix=4,
    beam_size=1,
):


    tags = read_tags(tags)
    dataset = read_tagged_sentences(dataset)
    params = pickle.load(open(model, 'rb'))
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=0
        )

    # Load model.
    model = LinearModel(params.values.shape[0])
    model.params().assign(params)

    # Tag all sentences.
    tagged_sentences = tag_sentences(dataset, <YOUR_PARAMS>)

    # Write tagged sentences.
    for tagged_sentence in tagged_sentences:
        write_tagged_sentence(tagged_sentence, sys.stdout)

    # Measure and print quality.
    q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dataset))
    print(q, file=sys.stderr)


In [ ]:
# test 
test(model='./model_beam_4.npz')

# sanity chec: accuracy > 90%.

### Part X: play with it

_This part is optional_

Once you've built something, it's only natural to test the limits of your contraption.

At minumum, we want you to find out how default model accuracy depends on __beam size__

To get maximum points, your model should get final quality >= 93% 

Any further analysis is welcome, as always.

In [ ]:
<YOUR CODE>